In [1]:
import core.utils.configuration

Config = core.utils.configuration.initialize_config()

with Config:
    Config.Data.Type = 'paired_images_with_contrast_label'
    Config.Data.Image.Preprocess = 'GrayImage' #'GrayImage' 'RGBImage' 'BGRImage' 
    Config.Data.Image.Postprocess = 'CLAHE_float' #'CLAHE_float' 'CLAHE_int'
    
    Config.Model.Task = 'SegWithClass'
    Config.Model.Param = dict(
        backbone_name = 'UniversalUnet', #'UniversalUnet' 'EffiNetB4Unet_AttDs' 'EffiNetB4Unet_AttRes'
        backbone_param = dict(
            encoder_name = 'resnet50',
            encoder_weights = 'imagenet',
            in_channels = 3,
            segmentation_dim = 4,
            classfy_dim = 8),
        seg_head_name = 'Head_1',
        seg_head_param = dict(in_channels=4),
        classify_heads_name = 'Head_1',
        classify_heads_param = dict(in_channels=8)
        )
    
    Config.Training.Settings.TrailNumber = 1
    Config.Training.Settings.Epochs = 30
    Config.Training.Settings.AmpScaleTrain = True
    
    Config.Training.Settings.Random.RootSeed = 100
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    
    Config.Training.Dataset.Path = r'D:\Datasets\Cardiac_Catheterization\train'
    Config.Training.Dataset.NumWorkers = 0
    Config.Training.Dataset.PinMemory = False
    Config.Training.Dataset.PrefetchFactor = 2
    Config.Training.Dataset.Train.BatchSize = 8
    Config.Training.Dataset.Val.Ratio = 0.1
    Config.Training.Dataset.Val.BatchSize = 16
    Config.Training.Dataset.Augmentation.Prob = 'Constant(0.8)'
    Config.Training.Dataset.Augmentation.Content = [
        'GridDistortion             -p:Constant(0.8)',
        'ElasticTransform           -p:Constant(0.8)',
        'Affine                     -p:Constant(0.8)',
        'GaussNoise                 -p:Constant(0.8)',
        'Blur                       -p:Constant(0.8)',
        'Downscale                  -p:Constant(0.8)',
        'RandomBrightnessContrast   -p:Constant(0.8)'
    ]
    
    Config.Training.Losses = [
        'BCELoss2D  -subject:"mask"           -weights:"contrast_exist" -schedule:WarmUp(warmup_epochs=5,max_value=2)  -smooth:1',
        'DiceLoss   -subject:"mask"           -weights:"contrast_exist" -schedule:Constant(1)',
        'BCELoss    -subject:"contrast_exist" -weights:None             -schedule:Constant(1)'
    ]
    
    Config.Training.Optimizer.Name = 'Adam'
    Config.Training.Optimizer.Param = {'lr':0.001}
    Config.Training.LearningRateSchedule.Name = ''
    Config.Training.LearningRateSchedule.Param = {}
    
    Config.Training.Metrics = [
        'BinaryAccuracy2D   -subject:"mask"           -weights:"contrast_exist" -threshold:0.5  -smooth:1',
        'DiceAccuracy       -subject:"mask"           -weights:"contrast_exist" -threshold:0.5',
        'BinaryAccuracy     -subject:"contrast_exist" -weights:None             -threshold:0.5'
    ]
    
    Config.Record.Frequence = 8
    Config.Record.RootPath = 'test_save_folder'
    Config.Record.SaveModelWeights = True
    Config.Record.Note = ''

In [ ]:
%run train2.py